# 这里将会介绍如何使用LLM进行接口编排调用



# 定义llm

In [1]:
from openai import OpenAI
import json
import re

client = OpenAI(api_key="your_zhipu_key", base_url="https://open.bigmodel.cn/api/paas/v4/")


def llm(messages):
    # 输入str 或者list 输入list时自动在后面追加生成的内容
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
    elif isinstance(messages, list):
        pass
    else:
        raise ValueError("输入格式错误")

    completion = client.chat.completions.create(
        model="GLM-4-Plus",
        messages=messages,
        top_p=0.1,
        max_tokens=5000,
        stream=True,
        tools=[{"type": "web_search", "web_search": {"enable": False, "type": "web_search"}}],
    )
    answer = ""
    for chunk in completion:
        if chunk.choices[0].delta:
            content = chunk.choices[0].delta.content
            if isinstance(c, str):
                answer += content

    if isinstance(messages, list):
        messages.append({"role": "assistant", "content": answer})
    return answer


def super_eval(json_str, try_num=0):
    # 辅助获取json的函数
    if try_num > 5:
        return "json格式错误"
    json_str = json_str.replace("：", ":")
    try:
        all_json = re.findall("```json(.*?)```", json_str, re.DOTALL)
        if all_json:
            try:
                return eval(all_json[-1])
            except:
                return json.loads(all_json[-1])
        if "```json" in json_str:
            json_str = json_str.replace("```json", "")
        json_str = json_str.replace("```", "")
        try:
            return eval(json_str)
        except:
            return json.loads(json_str)
    except:
        import traceback

        traceback.print_exc()
        text = llm(f"输出以下内容的json部分并修复成正确格式备注仅仅输出最后的json:```{json_str}```")
        try_num += 1
        return super_eval(text, try_num)

# 开始解决一个问题吧

这里将会给出一个简单的示例，在这个示例中，我会展示如何使用原生的大语言模型一步一步的获取正确答案。

In [3]:
question = "91310114312234914L对应的公司是否被限制高消费？如果是被限制高消费的最大涉案金额为（保留两位小数）？"

分析：在这个问题中，涉及到查询，排序（获取最大）两类问题，其中查询需要先通过 91310114312234914L 查询到公司名称

其中 91310114312234914L 为统一社会信用代码，那么便有了两种思路。

1. 通过观察当前的问题和已知条件，进行下一步规划
2. 先进性规划，然后一步一步执行

这两种思路都可以实现。

# 观察-执行模式

对于此类问题，如何进行第一步是非常重要的，这里有以下几种方案。

1. 输入题目和API文档，让AI选择执行,反复观察执行结果并给出新的执行步骤。
2. 从命名实体识别开始，通过命名实体识别召回当前可执行步骤，然后让AI选择并执行。

第一种方法非常适合tools，我使用第二种的原因是因为在本场景下可以免去做召回的步骤，并且非常节省token。


##  使用AI进行命名实体识别

In [4]:
ner_prompt = """
实体表格
--
公司名称
公司简称
公司代码
统一社会信用代码
法院名称
地址 
省份
城市
区县
案号
--

容易混淆的实体有:
公司名称和公司简称
公司名称为公司全称，格式为 行政区划+字号+行业+组织形式组成 例如：上海建工集团股份有限公司
公司简称一般为公司的简写 例如:上海建工，另外一些不规范的叫法，如上海建工公司，也要识别为公司简称
统一社会信用代码
18位数字或者字母组成，一般为91开头，例如：
913310007200456372
9165292274222840XN
公司代码：
指上市公司代码，一般是6位组成，
例如：
300674
600900
如果用户手误，一般是重复输入 300674->330000667744(识别为公司代码),这时你要识别成正确的
法院名称
例如：安徽省高级人民法院
法院代字，省份简称和省份简称+数字为法院代字
示例：皖01，京，川123
地址以及省市区：
如果有地址你应该直接返回地址，而不是从地址中寻找省市区。
西藏自治区那曲地区安多县帕那镇中路13号 -> 地址，不返回省市区
有没有在安徽省判决的案件 -> 省份:安徽省
律师事务所名称
江西心者律师事务所
河南良承事务所

日期：
2020年1月1日
2020-10-11

其中，案号，公司代码有可能不规则，提示用户不规则。

案号：
案号格式为：“(”+收案年度+“)”+法院代字+类型代字+案件编号+“号”。 其中()可以是中英文。年份为(2020)，类型代字有民初，民终，民申，执等。
例如:(2020)新0104民初1269号
    (2019)川01民终6937号

你需要做的是实体识别，因为用户的表述可能会不清楚，例如6位数字是公司代码，18位是统一社会信用代码。
你不需要回答问题，只是将问题中的
使用以下json格式回答,结果在```json ```内：
```json
{
    "实体": [
      {
        "名称": "实体名称1",
        "格式匹配为": "实体格式1"
      },
      {
        "名称": "实体名称2",
        "格式匹配为": "实体格式2"
      }
      // 可以有更多的实体
    ]
  }
```

题目：上海建工集团股份有限公司有多少家子公司？
示例：
```json
{
    "实体": [
      {
        "名称": "上海建工集团股份有限公司",
        "格式匹配为": "公司名称"
      }
    ]
}
```

题目：航天机电公司涉案总额？
示例：
```json
{
    "实体": [
      {
        "名称": "航天机电",
        "格式匹配为": "公司简称"
      }
    ]
}
```

题目：安利股份在子公司共投资了多少亿元
示例：
```json
{
"实体": [
  {
    "名称": "安利股份",
    "格式匹配为": "公司简称"
  }
}
```

题目：19年江苏省高级人民法院判，民申6268号，法院判决胜诉方是哪个公司？胜诉方律师事务所的地址在哪里？
示例：
```json
  {
    "实体": [
      {
        "名称": "江苏省高级人民法院",
        "格式匹配为": "法院名称"
      },
      {
        "名称": "19年江苏省高级人民法院判，民申6268号",
        "格式匹配为": "不规则案号"
      }
    ]
  }
```
题目:合肥市高新区科学大道91号地址对应的省市区县分别是
示例：
```json
  {
    "实体": [
      {
        "名称": "合肥市高新区科学大道91号",
        "格式匹配为": "地址"
      }
    ]
  }
```
"""

In [5]:
ner_messages = [{"role": "system", "content": ner_prompt}, {"role": "user", "content": question}]
ner_result = super_eval(llm(ner_messages))
ner_result

```json
{
    "实体": [
      {
        "名称": "91310114312234914L",
        "格式匹配为": "统一社会信用代码"
      }
    ]
}
```
{'completion_tokens': 43, 'prompt_tokens': 987, 'total_tokens': 1030}


{'实体': [{'名称': '91310114312234914L', '格式匹配为': '统一社会信用代码'}]}

##  开始第一次查询 

经过此次查询，我们得到的信息为，题目中只有一个统一社会信用代码是符合我们输入条件的参数，而此参数只对应一个接口，对于唯一的选择，我们当然可以写出执行规则，但是如果是多种选择，这时候就要交由AI进行编写代码。

假设我们通过一个字典，从统一社会信用代码找回到了相关的接口文档，然后使用llm进行编写代码。

In [6]:
doc = """
{'路由': '/get_company_register_name', '数据表': 'CompanyRegister',
         'API描述': '根据统一社会信用代码查询公司名称',
         '输入参数': {'query_conds': {'统一社会信用代码': 'str'}, 'need_fields': ['CompanyRegister属性值']},
         '参数示例': {'query_conds': {'统一社会信用代码': '913305007490121183'}, 'need_fields': []},
         '返回值示例': {'公司名称': '天能电池集团股份有限公司'}, '作用': '通过统一社会信用代码查询公司名称',
         '备注': '必须是统一社会信用代码'}
"""

code_prompt = """
# 以下是 API 文档
{}
-----
请你帮我实现接口调用代码：
请你基于API接口，构建查询的执行代码，具体来说时查询路由和查询参数构建查询代码。
你每次只需要返回当前需要的查询url和查询参数，用户会给你查询结果方便构建下一次查询参数。
我已经实现了python查询接口：
result = call_api(route: str, param: Dict, requirement: str) -> Dict or List[Dict]
当你返回python代码时，系统会执行代码并返回给你结果。
以下是一个通用示例：
```python
# 作用：这是一个代码示例
from api import call_api
route = '/<具体的路由>' -> str

query_conds = {{"key":"value"}}
need_fields = ['field1','field2','field3','field4','field5'] # 在这里列举你需要的字段，推荐3-5个
param = {{"query_conds":query_conds, "need_fields":need_fields}}  # need_fields字段不可省略可以为[]
requirement = "你简要的需求描述" # 用于日志
your_result_name = call_api(route,param,requirement) # 注意参数应该保存，请将your_result_name换成规范的名字 call_api接口会自动打印搜索内容
```
注意：
1.当你获取的结果需要进行批量操作的时候，也可以编写脚本，系统会执行并返回给你打印的结果，你在拿到返回结果的时候，会判断结果是否符合预期，
2.如果符合预期则输出结果，并提示任务已完成，如果不符合预期，或者接口提示错误，则按照提示修改代码。
3.代码在ipython中运行，所有之前运行的代码已经保存
4.查询代码执行成功后，如果有内容，请相信内容。
5.你每次只会完成提示的步骤内容，不要抢答。
6.不要使用假设参数，也不要在代码中出现假设这个词，会被拒绝运行。
7.如果查询成功，即使只要一条记录，请相信结果，因为这是内部数据库。
8.注意将结果命名成规范的名称
9.你会在代码的第一行注释代码的作用 格式为 ```python\n# 作用：xxx\n[你的代码]\n```
10.代码在ipython中运行，如果用到之前的参数，请直接引用不要定义，尤其是对于排序计算等类型。
"""

In [7]:
code_messages = [
    {"role": "system", "content": code_prompt.format(doc)},
    {"role": "user", "content": question + f"实体识别结果：{ner_result} 请开始编码"},
]
code_result = llm(code_messages)

```python
# 作用：根据统一社会信用代码查询公司名称
from api import call_api
route = '/get_company_register_name'
query_conds = {"统一社会信用代码": "91310114312234914L"}
need_fields = []
param = {"query_conds": query_conds, "need_fields": need_fields}
requirement = "查询公司名称"
company_info = call_api(route, param, requirement)
company_info
```
{'completion_tokens': 94, 'prompt_tokens': 752, 'total_tokens': 846}


##  运行函数

通过以上对话我们已经获取了一个可以执行的代码，接下来我们要做的是：

1. 使用正则表达式获取代码
2. 执行

执行代码当然可以使用 exec 函数，但是这样会伴随很多风险，为了使AI生成的代码在隔离的环境中运行，你可以写入文件然后执行，或者使用 ipython 进行执行，但是要做出一些防止风险的策略，例如设置权限。

以下是通过GLM4示例中改写的执行代码的示例

In [8]:
from kernel import run_code, CodeKernel

kernel = CodeKernel()
execute_result = run_code(code_result, kernel)
execute_result

Code kernel started.


"查询公司名称参数为：{'query_conds': {'统一社会信用代码': '91310114312234914L'}, 'need_fields': []} 查询结果为:{'公司名称': '上海红星美凯龙住建集采商贸有限公司'}\n"

## 连续查询

在上一步我们已经得到了一步结果，并且可以拿到  {'公司名称': '上海红星美凯龙住建集采商贸有限公司'} 这个字典，于是我们可以继续从接口文档中召回输入参数为统一社会信用代码和公司名称的接口,假设我们召回了如下三个API文档，便可以继续追问。

In [9]:
api_doc = r"""
{'路由': '/get_company_register', '数据表': 'CompanyRegister',
 'API描述': '根据公司名称，查询工商信息，注意/get_company_register接口无法根据统一社会信用代码查询公司名称',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['CompanyRegister属性值']},
 '参数示例': {'query_conds': {'公司名称': '天能电池集团股份有限公司'}, 'need_fields': []},
 '返回值示例': {'公司名称': '天能电池集团股份有限公司', '登记状态': '存续',
                '统一社会信用代码': '913305007490121183', '法定代表人': '杨建芬',
                '注册资本': '97210', '成立日期': '2003-03-13', '企业地址': '浙江省长兴县煤山镇工业园区',
                '联系电话': '0572-6029388',
                '联系邮箱': 'dshbgs@tiannenggroup.com', '注册号': '330500400001780',
                '组织机构代码': '74901211-8', '参保人数': '709',
                '行业一级': '制造业', '行业二级': '电气机械和器材制造业', '行业三级': '电池制造',
                '曾用名': '天能电池集团有限公司,\n浙江天能电池有限公司',
                '企业简介': '公司简介。',
                '经营范围': '经营范围'}
}

{'路由': '/get_sub_company_info', '数据表': 'SubCompanyInfo',
 'API描述': '根据被投资的子公司名称获得投资该公司的上市公司、投资比例、投资金额等信息',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['SubCompanyInfo属性值']},
 '参数示例': {'query_conds': {'公司名称': '上海爱斯达克汽车空调系统有限公司'},
              'need_fields': []},
 '返回值示例': {'关联上市公司全称': '上海航天汽车机电股份有限公司', '上市公司关系': '子公司',
                '上市公司参股比例': '87.5', '上市公司投资金额': '8.54亿',
                '公司名称': '上海爱斯达克汽车空调系统有限公司'},
 '作用': '查询特定子公司的投资信息，包括投资上市公司、比例和金额', '备注': '必须是被投资子公司的名称'},

{'路由': '/get_xzgxf_info_list', '数据表': 'XzgxfInfo', 'API描述': '根据企业名称查询所有限制高消费相关信息list',
 '输入参数': {"query_conds": {"限制高消费企业名称": "str"}, "need_fields": ['XzgxfInfo属性值']},
 '参数示例': {"query_conds": {"限制高消费企业名称": "欣水源生态环境科技有限公司"}, "need_fields": []},
 '返回值示例': {'限制高消费企业名称': '欣水源生态环境科技有限公司', '案号': '（2023）黔2731执恢130号',
                '法定代表人': '刘福云', '申请人': '四川省裕锦建设工程有限公司惠水分公司',
                '涉案金额': '7500000', '执行法院': '贵州省黔南布依族苗族自治州惠水县人民法院',
                '立案日期': '2023-08-04 00:00:00', '限高发布日期': '2023-11-09 00:00:00'},

         
"""

In [10]:
code_messages[0]["content"] = code_prompt.replace("{}", api_doc)
code_messages.append({"role": "user", "content": f"之前代码的运行结果为：{execute_result}，请编写下一步的执行代码"})
code_result = llm(code_messages)

```python
# 作用：根据公司名称查询是否被限制高消费及最大涉案金额
from api import call_api
route = '/get_xzgxf_info_list'
query_conds = {"限制高消费企业名称": "上海红星美凯龙住建集采商贸有限公司"}
need_fields = ['限制高消费企业名称', '涉案金额']
param = {"query_conds": query_conds, "need_fields": need_fields}
requirement = "查询是否被限制高消费及最大涉案金额"
xzgxf_info = call_api(route, param, requirement)
xzgxf_info
```
{'completion_tokens': 132, 'prompt_tokens': 1575, 'total_tokens': 1707}


In [11]:
execute_result = run_code(code_result, kernel)
execute_result

"查询是否被限制高消费及最大涉案金额参数为：{'query_conds': {'限制高消费企业名称': '上海红星美凯龙住建集采商贸有限公司'}, 'need_fields': ['限制高消费企业名称', '涉案金额', '限制高消费企业名称']} 查询结果为:[{'限制高消费企业名称': '上海红星美凯龙住建集采商贸有限公司', '涉案金额': '-'}, {'限制高消费企业名称': '上海红星美凯龙住建集采商贸有限公司', '涉案金额': '70185.85'}, {'限制高消费企业名称': '上海红星美凯龙住建集采商贸有限公司', '涉案金额': '699754.66'}] # 只显示前三条 一共19条，如果题目查看所有的子公司或者案号的需求，请你单独打印。\n"

## 接下来可以自由发挥了

获取的为一个列表，你可以实现一个获取第一名的函数，例如 def get_top1() 然后当成接口传入，当然，你也可以完全相信大模型的代码


In [12]:
code_messages.append(
    {
        "role": "user",
        "content": f"之前代码的运行结果为：{execute_result}，请编写下一步的执行代码，之前的参数已经保存，请继续使用之前的参数,并且使用print打印结果",
    }
)
code_result = llm(code_messages)

```python
# 作用：根据查询结果判断是否被限制高消费及计算最大涉案金额
xzgxf_info

# 提取并保留两位小数
amounts = [float(record['涉案金额']) for record in xzgxf_info if record['涉案金额'] != '-']
max_amount = max(amounts) if amounts else 0.00
max_amount = round(max_amount, 2)

# 打印结果
print(f'公司是否被限制高消费：{"是" if amounts else "否"}')
print(f'最大涉案金额为：{max_amount}')
```
{'completion_tokens': 131, 'prompt_tokens': 1942, 'total_tokens': 2073}


In [13]:
execute_result = run_code(code_result, kernel)
execute_result

'公司是否被限制高消费：是\n最大涉案金额为：2143812.0\n'

## 代码修复

有时AI编写的代码会报错，大多数情况下只需要返回报错信息即可。

In [ ]:
code_messages.append(
    {"role": "user", "content": f"之前代码的运行结果为：{execute_result}，请修复代码,并且使用print打印结果"}
)
code_result = llm(code_messages)

In [ ]:
execute_result = run_code(code_result, kernel)
execute_result

## 这样我们便简单完成了 观察-执行模式 的代码 , 接下来介绍 规划-执行模式
首先对于规划-执行，我们需要先定义一个json格式，保存我们的规划信息

在这里我设计的json格式为

```json
[
  {
    "step": 1,
    "goal": "获取所有相关案件信息",
    "type": "查询",
    "suggestion": "调用 /get_legal_document_list 接口，参数为 {'query_conds': {'关联公司': '沈阳先锋工程机械销售有限公司'}, 'need_fields': ['案号', '原告', '涉案金额']}",
    "table_name": "LegalDoc",
    "is_necessary": "necessary",
    "base_on_step": [0]
  },
  {
    "step": 2,
    "goal": "过滤出该公司作为原告的案件",
    "type": "过滤",
    "suggestion": "使用 filter_legal_docs 函数，参数为 {'原告': '沈阳先锋工程机械销售有限公司'}",
    "table_name": "LegalDoc",
    "is_necessary": "necessary",
    "base_on_step": [1]
  },
  {
    "step": 3,
    "goal": "根据涉案金额对案件进行排序,并找出涉案金额第二高的案件",
    "type": "计算",
    "suggestion": "调用 /rank 接口，参数为 {'data_list': 案件列表, 'key': '涉案金额', 'is_desc': True}" 然后取出涉案金额第二高的案件信息,
    "table_name": "LegalDoc",
    "is_necessary": "necessary",
    "base_on_step": [2]
  }]
```

这样的好处为：使用json对象固定了格式，减少了文本解析的工作量，以下是具体的实现方法。

In [41]:
plan_prompt = """

# 以下是API表

{'路由': '/get_company_register_name', '数据表': 'CompanyRegister',
         'API描述': '根据统一社会信用代码查询公司名称',
         '输入参数': {'query_conds': {'统一社会信用代码': 'str'}, 'need_fields': ['CompanyRegister属性值']},
         '参数示例': {'query_conds': {'统一社会信用代码': '913305007490121183'}, 'need_fields': []},
         '返回值示例': {'公司名称': '天能电池集团股份有限公司'}, '作用': '通过统一社会信用代码查询公司名称',
         '备注': '必须是统一社会信用代码'}
{'路由': '/get_company_register', '数据表': 'CompanyRegister',
 'API描述': '根据公司名称，查询工商信息，注意/get_company_register接口无法根据统一社会信用代码查询公司名称',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['CompanyRegister属性值']},
 '参数示例': {'query_conds': {'公司名称': '天能电池集团股份有限公司'}, 'need_fields': []},
 '返回值示例': {'公司名称': '天能电池集团股份有限公司', '登记状态': '存续',
                '统一社会信用代码': '913305007490121183', '法定代表人': '杨建芬',
                '注册资本': '97210', '成立日期': '2003-03-13', '企业地址': '浙江省长兴县煤山镇工业园区',
                '联系电话': '0572-6029388',
                '联系邮箱': 'dshbgs@tiannenggroup.com', '注册号': '330500400001780',
                '组织机构代码': '74901211-8', '参保人数': '709',
                '行业一级': '制造业', '行业二级': '电气机械和器材制造业', '行业三级': '电池制造',
                '曾用名': '天能电池集团有限公司,\n浙江天能电池有限公司',
                '企业简介': '公司简介。',
                '经营范围': '经营范围'}
}

{'路由': '/get_sub_company_info', '数据表': 'SubCompanyInfo',
 'API描述': '根据被投资的子公司名称获得投资该公司的上市公司、投资比例、投资金额等信息',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['SubCompanyInfo属性值']},
 '参数示例': {'query_conds': {'公司名称': '上海爱斯达克汽车空调系统有限公司'},
              'need_fields': []},
 '返回值示例': {'关联上市公司全称': '上海航天汽车机电股份有限公司', '上市公司关系': '子公司',
                '上市公司参股比例': '87.5', '上市公司投资金额': '8.54亿',
                '公司名称': '上海爱斯达克汽车空调系统有限公司'},
 '作用': '查询特定子公司的投资信息，包括投资上市公司、比例和金额', '备注': '必须是被投资子公司的名称'},

{'路由': '/get_xzgxf_info_list', '数据表': 'XzgxfInfo', 'API描述': '根据企业名称查询所有限制高消费相关信息list',
 '输入参数': {"query_conds": {"限制高消费企业名称": "str"}, "need_fields": ['XzgxfInfo属性值']},
 '参数示例': {"query_conds": {"限制高消费企业名称": "欣水源生态环境科技有限公司"}, "need_fields": []},
 '返回值示例': {'限制高消费企业名称': '欣水源生态环境科技有限公司', '案号': '（2023）黔2731执恢130号',
                '法定代表人': '刘福云', '申请人': '四川省裕锦建设工程有限公司惠水分公司',
                '涉案金额': '7500000', '执行法院': '贵州省黔南布依族苗族自治州惠水县人民法院',
                '立案日期': '2023-08-04 00:00:00', '限高发布日期': '2023-11-09 00:00:00'},
注意：need_fields字段为对应表格属性，不可省略，填[]获取所有字段
-----

一般常用的解题路径为：
公司简称/统一社会信用代码/公司代码 -> 公司名称 -> LegalDoc.关联公司/XzgxfInfo.限制高消费企业名称 -> 过滤筛选等操作 -> 案号及案件信息列表 -> 计算操作
案号 -> 法院名称 -> 法院级别/官网/地址
案号 -> 原告/被告/原告律师事务所/被告律师事务所/审理法院/执行法院(XzgxfInfo) -> 获取所在实体表中地址 -> 区划代码/天气(获取天气不要忘记获取日期)
公司简称/统一社会信用代码 -> 公司全称 -> 子公司信息
子公司名称 -> 母公司名称(关联上市公司全称) -> 对母公司进行各种操作
子公司名称 -> SubCompanyInfo -> 计算被投资总额（投资额度➗控股比例）

用户会给你一个问题，，将用户的问题转化为一连串的查询计算问题，
查询工具使用的只有python调用上面的API
然后你会给用户一个执行规划，规划如何使用API完成上面的问题，
最后返回json格式的执行规划。

# 注意事项：
1.只考虑能使用API进行操作的查询,并使用尽量少的查询步骤。
2.根据用户提供的数据，不要编造示例数据。
3.给出具体的操作流程，每一个流程目标明确，但是方法应该包含多种选择。
4.接口没有过滤功能，过滤操作为先获取全量数据，然后使用辅助函数，注意使用辅助函数时，请仔细阅读 function doc，参数应该保持最少原则。
5.排序，求和，找第几的为计算操作，使用 /sum /rank 接口

请按照以下格式提供输出：
推理流程：
```md
[分析解题目标，明确题目需求]
[分析解题路径，需要说出是用第几步的结果，参考解题路径]
[分析参数，用户提供的问题中的实体每对应哪张表的哪个字段，输入参数，结果参数，中间参数是什么]
```
执行步骤：
```json
[
 {{"step":1,"goal":"xxx","type":"查询/计算/过滤/统计/列举","base_on_step":[int]}},
 {{"step":2,"goal":"xxx","type":"查询/计算/过滤/统计/列举","base_on_step":[int]}}
  ...
]
```
注意：1.step为当前步骤，为1，2，3递增，请确保输出正确的json格式，步骤应该互相独立。
     2.[查询/计算/过滤]不能混在一个步骤，例如过滤后求和，应该有过滤和计算两个步骤。
     3.base_on_step:这一步的输入是基于哪几步的结果，填0表示基于题目本身。
     4./get_sum 不能统计数量，统计数量 直接用 len(之前的结果)
"""

In [42]:
plan_messages = [{"role": "system", "content": plan_prompt}, {"role": "user", "content": question + "请输出json"}]
plan_result = llm(plan_messages)

推理流程：
```md
1. 我们需要根据用户提供的统一社会信用代码（91310114312234914L）查询公司名称。
2. 通过公司名称查询该公司的所有限制高消费相关信息。
3. 对限制高消费信息进行筛选，如果存在限制高消费信息，则找到最大涉案金额。
```

执行步骤：
```json
[
  {
    "step": 1,
    "goal": "根据统一社会信用代码查询公司名称",
    "type": "查询",
    "base_on_step": [0],
    "api": "/get_company_register_name",
    "parameters": {
      "query_conds": {
        "统一社会信用代码": "91310114312234914L"
      },
      "need_fields": []
    }
  },
  {
    "step": 2,
    "goal": "查询该公司的所有限制高消费相关信息",
    "type": "查询",
    "base_on_step": [1],
    "api": "/get_xzgxf_info_list",
    "parameters": {
      "query_conds": {
        "限制高消费企业名称": "{{step1.公司名称}}"
      },
      "need_fields": []
    }
  },
  {
    "step": 3,
    "goal": "过滤并找到最大涉案金额",
    "type": "计算",
    "base_on_step": [2],
    "api": "/get_sum",
    "parameters": {
      "data": "{{step2}}",
      "field": "涉案金额",
      "operation": "max",
      "precision": 2
    }
  }
]
```
{'completion_tokens': 351, 'prompt_tokens': 1632, 'total_tokens': 1983}


##  获取执行步骤

在获取了上述推理流程后，我们便可以使用 super_eval 获取json并且循环输入AI即可

In [43]:
plan_list = super_eval(plan_result)
plan_list

[{'step': 1,
  'goal': '根据统一社会信用代码查询公司名称',
  'type': '查询',
  'base_on_step': [0],
  'api': '/get_company_register_name',
  'parameters': {'query_conds': {'统一社会信用代码': '91310114312234914L'},
   'need_fields': []}},
 {'step': 2,
  'goal': '查询该公司的所有限制高消费相关信息',
  'type': '查询',
  'base_on_step': [1],
  'api': '/get_xzgxf_info_list',
  'parameters': {'query_conds': {'限制高消费企业名称': '{{step1.公司名称}}'},
   'need_fields': []}},
 {'step': 3,
  'goal': '过滤并找到最大涉案金额',
  'type': '计算',
  'base_on_step': [2],
  'api': '/get_sum',
  'parameters': {'data': '{{step2}}',
   'field': '涉案金额',
   'operation': 'max',
   'precision': 2}}]

In [44]:
execute_prompt = """
# 以下是API表

{'路由': '/get_company_register_name', '数据表': 'CompanyRegister',
         'API描述': '根据统一社会信用代码查询公司名称',
         '输入参数': {'query_conds': {'统一社会信用代码': 'str'}, 'need_fields': ['CompanyRegister属性值']},
         '参数示例': {'query_conds': {'统一社会信用代码': '913305007490121183'}, 'need_fields': []},
         '返回值示例': {'公司名称': '天能电池集团股份有限公司'}, '作用': '通过统一社会信用代码查询公司名称',
         '备注': '必须是统一社会信用代码'}
{'路由': '/get_company_register', '数据表': 'CompanyRegister',
 'API描述': '根据公司名称，查询工商信息，注意/get_company_register接口无法根据统一社会信用代码查询公司名称',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['CompanyRegister属性值']},
 '参数示例': {'query_conds': {'公司名称': '天能电池集团股份有限公司'}, 'need_fields': []},
 '返回值示例': {'公司名称': '天能电池集团股份有限公司', '登记状态': '存续',
                '统一社会信用代码': '913305007490121183', '法定代表人': '杨建芬',
                '注册资本': '97210', '成立日期': '2003-03-13', '企业地址': '浙江省长兴县煤山镇工业园区',
                '联系电话': '0572-6029388',
                '联系邮箱': 'dshbgs@tiannenggroup.com', '注册号': '330500400001780',
                '组织机构代码': '74901211-8', '参保人数': '709',
                '行业一级': '制造业', '行业二级': '电气机械和器材制造业', '行业三级': '电池制造',
                '曾用名': '天能电池集团有限公司,\n浙江天能电池有限公司',
                '企业简介': '公司简介。',
                '经营范围': '经营范围'}
}

{'路由': '/get_sub_company_info', '数据表': 'SubCompanyInfo',
 'API描述': '根据被投资的子公司名称获得投资该公司的上市公司、投资比例、投资金额等信息',
 '输入参数': {'query_conds': {'公司名称': 'str'}, 'need_fields': ['SubCompanyInfo属性值']},
 '参数示例': {'query_conds': {'公司名称': '上海爱斯达克汽车空调系统有限公司'},
              'need_fields': []},
 '返回值示例': {'关联上市公司全称': '上海航天汽车机电股份有限公司', '上市公司关系': '子公司',
                '上市公司参股比例': '87.5', '上市公司投资金额': '8.54亿',
                '公司名称': '上海爱斯达克汽车空调系统有限公司'},
 '作用': '查询特定子公司的投资信息，包括投资上市公司、比例和金额', '备注': '必须是被投资子公司的名称'},

{'路由': '/get_xzgxf_info_list', '数据表': 'XzgxfInfo', 'API描述': '根据企业名称查询所有限制高消费相关信息list',
 '输入参数': {"query_conds": {"限制高消费企业名称": "str"}, "need_fields": ['XzgxfInfo属性值']},
 '参数示例': {"query_conds": {"限制高消费企业名称": "欣水源生态环境科技有限公司"}, "need_fields": []},
 '返回值示例': {'限制高消费企业名称': '欣水源生态环境科技有限公司', '案号': '（2023）黔2731执恢130号',
                '法定代表人': '刘福云', '申请人': '四川省裕锦建设工程有限公司惠水分公司',
                '涉案金额': '7500000', '执行法院': '贵州省黔南布依族苗族自治州惠水县人民法院',
                '立案日期': '2023-08-04 00:00:00', '限高发布日期': '2023-11-09 00:00:00'},
注意：need_fields字段为对应表格属性，不可省略，填[]获取所有字段
请你帮我实现接口调用代码：
请你基于API接口，构建查询的执行代码，具体来说时查询路由和查询参数构建查询代码。
你每次只需要返回当前需要的查询url和查询参数，用户会给你查询结果方便构建下一次查询参数。
我已经实现了python查询接口：
result = call_api(route: str, param: Dict, requirement: str) -> Dict or List[Dict]
当你返回python代码时，系统会执行代码并返回给你结果。
以下是一个通用示例：
```python
# 作用：这是一个代码示例
from api import call_api
route = '/<具体的路由>' -> str

query_conds = {{"key":"value"}}
need_fields = ['field1','field2','field3','field4','field5'] # 在这里列举你需要的字段，推荐3-5个
param = {{"query_conds":query_conds, "need_fields":need_fields}}  # need_fields字段不可省略可以为[]
requirement = "你简要的需求描述" # 用于日志
your_result_name = call_api(route,param,requirement) # 注意参数应该保存，请将your_result_name换成规范的名字 call_api接口会自动打印搜索内容
```
注意：
1.当你获取的结果需要进行批量操作的时候，也可以编写脚本，系统会执行并返回给你打印的结果，你在拿到返回结果的时候，会判断结果是否符合预期，
2.如果符合预期则输出结果，并提示任务已完成，如果不符合预期，或者接口提示错误，则按照提示修改代码。
3.代码在ipython中运行，所有之前运行的代码已经保存
4.查询代码执行成功后，如果有内容，请相信内容。
5.你每次只会完成提示的步骤内容，不要抢答。
6.不要使用假设参数，也不要在代码中出现假设这个词，会被拒绝运行。
7.如果查询成功，即使只要一条记录，请相信结果，因为这是内部数据库。
8.注意将结果命名成规范的名称
9.你会在代码的第一行注释代码的作用 格式为 ```python\n# 作用：xxx\n[你的代码]\n```
10.代码在ipython中运行，如果用到之前的参数，请直接引用不要定义，尤其是对于排序计算等类型。
"""

In [49]:
messages = [{"role": "system", "content": execute_prompt}]
kernel = CodeKernel()
for i in plan_list:
    if i["type"] == "计算":  # 可以根据不同的任务选择不同的提示
        messages.append(
            {"role": "user", "content": f"请完成目标：{i},注意使用之前保存的参数，不要重新定义，注意结果使用print打印"}
        )
    else:
        messages.append({"role": "user", "content": f"请完成目标：{i}"})
    code_result = llm(messages)
    execute_result = run_code(code_result, kernel)
    for _ in range(3):  # 可以选择修复代码
        if "Error" in code_result:
            messages.append({"role": "user", "content": f"代码执行结果为：{execute_result}，请修复代码"})
            execute_result = run_code(code_result, kernel)
        else:
            messages.append({"role": "user", "content": f"代码执行结果为：{execute_result},请等待下一步指示"})
            messages.append({"role": "assistant", "content": f"请指示下一步"})
            break

messages.append({"role": "user", "content": f"基于历史对话，使用自然语言回答问题`{question}`"})
llm(messages)

Code kernel started.
```python
# 作用：根据统一社会信用代码查询公司名称
from api import call_api

route = '/get_company_register_name'
query_conds = {'统一社会信用代码': '91310114312234914L'}
need_fields = []
param = {'query_conds': query_conds, 'need_fields': need_fields}
requirement = "根据统一社会信用代码查询公司名称"

company_name_result = call_api(route, param, requirement)
```
{'completion_tokens': 97, 'prompt_tokens': 1559, 'total_tokens': 1656}
```python
# 作用：查询上海红星美凯龙住建集采商贸有限公司的所有限制高消费相关信息
from api import call_api

route = '/get_xzgxf_info_list'
query_conds = {'限制高消费企业名称': '上海红星美凯龙住建集采商贸有限公司'}
need_fields = []
param = {'query_conds': query_conds, 'need_fields': need_fields}
requirement = "查询该公司的所有限制高消费相关信息"

xzgxf_info_result = call_api(route, param, requirement)
```
{'completion_tokens': 122, 'prompt_tokens': 1827, 'total_tokens': 1949}
```python
# 作用：过滤并找到最大涉案金额
import re

# 从上一步的结果中提取涉案金额，并将其转换为浮点数
涉案金额_list = []
for record in xzgxf_info_result:
    if record['涉案金额'] != '-':
        amount = float(re.sub('[,]', '', 

'统一社会信用代码为 `91310114312234914L` 的公司是上海红星美凯龙住建集采商贸有限公司。该公司有被限制高消费的记录，最大涉案金额为 2143812.00 元。'